# Querying Documentum from a Jupyter Notebook
_(c) 2019 MSRoth_

This notebook demostrates how to interface with the Documentum Content Server from a Jupyter Notebook using a Java kernel and the DFC.  Enter any valid query below and see the results rendered inline.

This notebook utilizes the iJava Kernel (https://github.com/SpencerPark/IJava), the Documentum Foundation Classes (DFC), and two helper classes:  DCTMBasics (https://github.com/msroth/dmBasics), and dmRecordSet (https://github.com/msroth/dmRecordSet).



_Use some Jupyter magic to set classpath.  Specifically, add the location of the dfc.properties file, the dctm.jar file, and the two helper classes to the classpath.  Adjust these paths as required for your environment._

In [1]:
%classpath "C:/Documentum/config"
%classpath "C:/Program Files/Documentum/dctm.jar"
%classpath "C:/users/rothmi/Documents/JupyterNotebooks/DCTM/*.jar"

import com.dm_misc.dctm.DCTMBasics;
import com.dm_misc.collections.dmRecordSet;
import com.documentum.fc.client.*;
import com.documentum.fc.common.*;

In [2]:
// Enter username, password, and docbase values below.
String user = "dmadmin";
String password = "dmadmin";
String docbase = "repo1";

// It takes a few seconds to logon... wait for session Id before continuing
System.out.print("Logging on...");
IDfSession session = DCTMBasics.logon(docbase, user, password);
if (session != null) { 
    System.out.println("Session Id: " + session.getSessionId());
} else {
    System.out.println("Could not logon");
}

Logging on...Session Id: s0


In [3]:
// Enter a valid query string
String dql = "select * from dm_sysobject where folder('/Temp')";

In [4]:
// Run the query
System.out.println("Running query...");

IDfCollection col = DCTMBasics.runSelectQuery(dql, session);
dmRecordSet dmRS = new dmRecordSet(col);
col.close();

if (dmRS.isEmpty()) {
    System.out.println("Query returned 0 objects.");
} else {
    System.out.println("Found " + dmRS.getRowCount() + " objects.");
}

// Render the results
if (!dmRS.isEmpty()) {

    // build a single output string with all results
    String output = "<table><tr>";
    
    // get all column names in the record set
    ArrayList<IDfAttr> cols = dmRS.getColumnDefs();

    // print col names as headers
    for (IDfAttr a : cols) {
        output += "<th>" + a.getName() + "</th>";
    }
    output += "</tr>";

    // print record set content under each column heading
    while (dmRS.hasNext()) {
        output += "<tr>";
        IDfTypedObject tObj = dmRS.getNextRow();
        for (IDfAttr a : cols) {
            output += "<td>" + tObj.getString(a.getName()) + "</td>";
        }
        output += "</tr>";
    }
    output += "</table>";
    
    // render the results as an inline HTML table
    display(output, "text/html");
}

Running query...
Found 13 objects.


r_object_id,object_name,title,subject,resolution_label,owner_name,owner_permit,group_name,group_permit,world_permit,log_entry,acl_domain,acl_name,language_code,r_object_type,r_creation_date,r_modify_date,a_content_type
0901e4538005895c,sample_news_story.docx,,,,dmadmin,7,admingroup,1,6,,dmadmin,dm_4501e45380014106,,dm_document,13.09.2019 13:44:23,13.09.2019 14:28:07,msw12
0901e4538005895d,sample_news_story.pdf,,,,dmadmin,7,admingroup,1,6,,dmadmin,dm_4501e45380014107,,dm_document,13.09.2019 13:44:23,13.09.2019 14:28:07,pdf
0901e4538005895e,test.html,,,,dmadmin,7,admingroup,1,6,,dmadmin,dm_4501e45380014108,,dm_document,13.09.2019 13:44:24,13.09.2019 14:28:08,html
0901e4538005895f,HelloWorld.rtf,,,,dmadmin,7,admingroup,1,6,,dmadmin,dm_4501e45380014109,,dm_document,13.09.2019 13:44:24,13.09.2019 14:28:07,rtf
0901e45380059540,Scott doc,,,,dmadmin,7,admingroup,1,6,,repo1,dm_4501e45380000103,,sr_document,18.09.2019 13:16:21,18.09.2019 13:16:52,pdf
0b01e45380000cee,Jobs,,,,dmadmin,7,docu,5,3,,dmadmin,dm_4501e45380000101,,dm_folder,06.05.2013 11:35:41,06.05.2013 11:35:41,
0b01e4538000450d,SIDocs,,,,dmadmin,7,docu,5,3,,dmadmin,dm_4501e45380000101,,dm_folder,03.07.2013 16:01:13,05.07.2013 16:15:53,
0b01e45380057e93,Import,,,,dmadmin,7,admingroup,1,6,,repo1,dm_4501e45380000103,,dm_folder,10.09.2019 12:38:24,10.09.2019 12:38:24,
0b01e45380058500,BPM,BPM Folder,Folder containing bpm-related objects.,,repo1,7,docu,5,6,,repo1,dm_4501e45380013d00,,dm_folder,12.09.2019 07:50:01,12.09.2019 07:50:02,
0b01e45380058514,Export2,,,,dmadmin,7,admingroup,1,6,,repo1,dm_4501e45380000103,,dm_folder,12.09.2019 11:56:42,12.09.2019 11:56:43,


_You can run multiple queries by changing the __dql__ variable and rerunning the cell above.  When you are finished, run the cell below to log off of the Documentum server._

In [5]:
// close session (logout)
if (session != null) {
    DCTMBasics.logoff(session);
    session = null;
}
System.out.println("Logged off");

Logged off
